In [3]:
# train_model_online_estimate_v3_adapter.py
import os
import time
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_absolute_error, classification_report, roc_auc_score, r2_score
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Concatenate, Conv1D, MaxPooling1D,
                                     Dense, Dropout, Lambda, BatchNormalization,
                                     GlobalAveragePooling1D, GlobalAveragePooling2D,
                                     Conv2D, MaxPooling2D, Reshape)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.losses import BinaryCrossentropy, SparseCategoricalCrossentropy, MeanSquaredError
from tensorflow.keras.metrics import BinaryAccuracy, SparseCategoricalAccuracy
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt

# ---------- 中文字体 ----------
def set_chinese_font():
    try:
        font_paths = ['/usr/share/fonts/truetype/wqy/wqy-microhei.ttc',
                      'C:/Windows/Fonts/simhei.ttf',
                      '/System/Library/Fonts/PingFang.ttc']
        for font_path in font_paths:
            if os.path.exists(font_path):
                fm.fontManager.addfont(font_path)
                plt.rcParams['font.family'] = fm.FontProperties(fname=font_path).get_name()
                plt.rcParams['axes.unicode_minus'] = False
                print(f"✅ 成功设置中文字体: {plt.rcParams['font.family']}")
                return True
        plt.rcParams['font.family'] = ['SimHei', 'Arial Unicode MS']
        plt.rcParams['axes.unicode_minus'] = False
        print("⚠️ 未找到指定字体，使用默认兼容字体")
        return True
    except Exception as e:
        print(f"❌ 字体设置失败: {e}")
        return False
set_chinese_font()

# ---------- 加载数据集 ----------
def load_dataset(npz_path="/root/yxun/20250826/dataset/interference_signals_natural_same_freq_1019.npz"):
    """加载数据集并适配v3版归一化参数"""
    data = np.load(npz_path, allow_pickle=True)
    signals = data["signals"]
    labels = data["labels"].astype(np.int32)
    jnr_values = data["jnr_values"].astype(np.float32)
    fs = float(data["fs"])
    L = int(data["L"])
    noise_power_db = float(data["noise_power_db"])
    type2label = data["type_to_label"].item()
    
    # 从metadata中提取参数统计信息用于反归一化
    metadata = data["metadata"] if "metadata" in data else None
    param_stats = calculate_param_statistics(metadata, L, fs)
    
    interference_type_names = {
        "satellite_signal": "Satellite_Signal",
        "single_tone": "Single_Tone",
        "comb_spectra": "Comb_Spectra",
        "sweeping": "Sweeping-LFM",
        "pulse": "Pulse",
        "frequency_hopping": "Frequency_Hopping",
        "noise_fm": "Noise_FM",
        "noise_am": "Noise_AM",
        "random_combination": "Random_Combination"
    }
    label2name = {i: interference_type_names[k] for k, i in type2label.items()}

    return {
        "signals": signals,
        "labels": labels,
        "jnr_values": jnr_values,
        "fs": fs,
        "L": L,
        "noise_power_db": noise_power_db,
        "type2label": type2label,
        "label2name": label2name,
        "param_stats": param_stats  # 新增：参数统计信息
    }

def calculate_param_statistics(metadata, L, fs):
    """从metadata计算参数统计信息用于反归一化"""
    if metadata is None:
        return {"total_duration_ms": L/fs*1e3, "jnr_range": [-10, 30]}
    
    # 提取所有参数用于统计
    all_start_times = []
    all_end_times = []
    all_jnr_values = []
    
    for m in metadata:
        params = m.get("params", {})
        if "start_time" in params:
            all_start_times.append(params["start_time"])
        if "end_time" in params:
            all_end_times.append(params["end_time"])
        if "jnr_db" in params:
            all_jnr_values.append(params["jnr_db"])
    
    # 计算统计量
    total_duration_ms = L / fs * 1e3  # 信号总时长(ms)
    
    return {
        "total_duration_ms": total_duration_ms,
        "start_time_mean": np.mean(all_start_times) if all_start_times else 0.5,
        "end_time_mean": np.mean(all_end_times) if all_end_times else 0.5,
        "jnr_mean": np.mean(all_jnr_values) if all_jnr_values else 10.0,
        "start_time_std": np.std(all_start_times) if all_start_times else 0.2,
        "end_time_std": np.std(all_end_times) if all_end_times else 0.2,
        "jnr_std": np.std(all_jnr_values) if all_jnr_values else 10.0
    }

# ---------- 数据预处理（适配v3归一化） ----------
def preprocess_data(dataset, aug_ratio=0.1):
    signals = dataset["signals"]
    labels = dataset["labels"]
    jnr_values = dataset["jnr_values"]
    L = dataset["L"]
    param_stats = dataset["param_stats"]

    # 信号标准化
    signals = np.array([StandardScaler().fit_transform(s.reshape(-1, 1)).ravel() for s in signals])

    # 检测标签
    no_key = "satellite_signal"
    det_labels = (labels != dataset["type2label"][no_key]).astype(np.float32)

    # 使用生成时记录的参数作为标签（v3版已归一化到[0,1]）
    param_labels = []
    if "metadata" in dataset and dataset["metadata"] is not None:
        for m in dataset["metadata"]:
            p = m.get("params", {})
            # v3版参数已经是归一化值，直接使用
            start_time = float(p.get("start_time", 0.0))   # ∈ [0,1]
            end_time = float(p.get("end_time", 0.0))     # ∈ [0,1]
            jnr_db = float(p.get("jnr_db", 0.0))          # 真实dB值
            
            param_labels.append([start_time, end_time, jnr_db])
    else:
        print("⚠️ 未找到 metadata，使用默认参数标签")
        param_labels = [[0.5, 0.5, 0.0] for _ in range(len(signals))]  # 使用中间值
    
    param_labels = np.array(param_labels, dtype=np.float32)

    # 对JNR进行归一化（与其他时间参数尺度一致）
    jnr_min, jnr_max = -10, 30  # JNR范围
    param_labels[:, 2] = (param_labels[:, 2] - jnr_min) / (jnr_max - jnr_min)  # 归一化到[0,1]

    # 丢弃含 NaN/Inf 的样本
    mask = ~(
        np.any(np.isnan(signals), axis=1) |
        np.any(np.isinf(signals), axis=1) |
        np.isnan(det_labels) | np.isinf(det_labels) |
        np.isnan(labels) | np.isinf(labels) |
        np.any(np.isnan(param_labels), axis=1) | np.any(np.isinf(param_labels), axis=1)
    )
    print(f"🧹 丢弃 {np.sum(~mask)} / {len(mask)} 条含 NaN/Inf 的样本")
    signals, det_labels, labels, param_labels, jnr_values = \
        signals[mask], det_labels[mask], labels[mask], param_labels[mask], jnr_values[mask]

    # 数据集分割
    X_train, X_tmp, y_det_train, y_det_tmp, y_type_train, y_type_tmp, y_param_train, y_param_tmp, jnr_train, jnr_tmp = train_test_split(
        signals, det_labels, labels, param_labels, jnr_values,
        test_size=0.3, random_state=42, stratify=labels
    )
    X_val, X_test, y_det_val, y_det_test, y_type_val, y_type_test, y_param_val, y_param_test, jnr_val, jnr_test = train_test_split(
        X_tmp, y_det_tmp, y_type_tmp, y_param_tmp, jnr_tmp,
        test_size=2/3, random_state=42, stratify=y_type_tmp
    )

    return {
        "X_train": X_train, "X_val": X_val, "X_test": X_test,
        "y_det_train": y_det_train, "y_det_val": y_det_val, "y_det_test": y_det_test,
        "y_type_train": y_type_train, "y_type_val": y_type_val, "y_type_test": y_type_test,
        "y_param_train": y_param_train, "y_param_val": y_param_val, "y_param_test": y_param_test,
        "jnr_values_train": jnr_train, "jnr_values_val": jnr_val, "jnr_values_test": jnr_test,
        "type2label": dataset["type2label"], "label2name": dataset["label2name"],
        "L": L, "fs": dataset["fs"], "noise_power_db": dataset["noise_power_db"],
        "param_stats": param_stats,  # 传递参数统计信息
        "jnr_range": [jnr_min, jnr_max]  # JNR范围用于反归一化
    }

# ---------- 反归一化函数 ----------
def denormalize_params(normalized_params, param_stats, jnr_range):
    """将归一化的参数反归一化为原始物理量"""
    total_duration_ms = param_stats["total_duration_ms"]
    jnr_min, jnr_max = jnr_range
    
    denormalized = np.zeros_like(normalized_params)
    # 反归一化开始时间：归一化值 → 毫秒
    denormalized[:, 0] = normalized_params[:, 0] * total_duration_ms
    # 反归一化结束时间：归一化值 → 毫秒  
    denormalized[:, 1] = normalized_params[:, 1] * total_duration_ms
    # 反归一化JNR：归一化值 → 原始dB值
    denormalized[:, 2] = normalized_params[:, 2] * (jnr_max - jnr_min) + jnr_min
    
    return denormalized

# ---------- 非均匀掩膜 ----------
@tf.function
def adaptive_diff_mask(x, n_levels=3):
    L = tf.shape(x)[0]
    seg_min = 2 ** n_levels
    L_pad = tf.cast(tf.math.ceil(L / seg_min) * seg_min, tf.int32)
    x_pad = tf.pad(x, [[0, L_pad - L]])
    x_avg = tf.squeeze(
        tf.nn.avg_pool1d(
            tf.expand_dims(tf.expand_dims(x_pad, 0), -1),
            ksize=seg_min, strides=seg_min, padding='VALID', data_format='NWC'
        ),
        [0, -1]
    )
    diff = x_avg[1:] - x_avg[:-1]
    flat_idx = tf.argmin(tf.abs(diff))
    n_seg = tf.shape(x_avg)[0]
    mask = tf.ones(n_seg, dtype=tf.float32)
    mask = tf.tensor_scatter_nd_update(mask, [[flat_idx]], [0.0])
    mask = tf.repeat(mask, seg_min)[:L_pad]
    return mask[:L]

# ---------- 数据增强 ----------
@tf.function
def aug_fn(x):
    x = tf.cast(x, tf.float32)
    if tf.random.uniform([]) > 0.2:
        snr = tf.random.uniform([], 5., 25.)
        noise = tf.random.normal(tf.shape(x), dtype=tf.float32) * tf.math.reduce_std(x) * tf.cast(10.0 ** (-snr / 20.0), tf.float32)
        x = x + noise
    if tf.random.uniform([]) > 0.3:
        shift = tf.random.uniform([], -100, 100, dtype=tf.int32)
        x = tf.roll(x, shift=shift, axis=0)
    if tf.random.uniform([]) > 0.3:
        scale = tf.random.uniform([], 0.7, 1.3)
        x = x * scale
    if tf.random.uniform([]) > 0.7:
        freq_shift = tf.random.uniform([], -0.1, 0.1)
        n = tf.cast(tf.shape(x)[0], tf.float32)
        x = x * tf.cos(2 * np.pi * freq_shift * tf.range(n, dtype=tf.float32))
    return x

@tf.function
def aug_fn_with_mask(x):
    x = aug_fn(x)
    mask = adaptive_diff_mask(x)
    x = x * mask
    return x

# ---------- 网络架构（适配v3归一化） ----------
def build_model(input_shape, num_classes):
    inputs = Input(shape=input_shape, dtype=tf.float32)
    x1 = Reshape((input_shape[0], 1))(inputs)
    x1 = Conv1D(64, 7, activation='relu', padding='same')(x1)
    x1 = BatchNormalization()(x1)
    x1 = MaxPooling1D(2)(x1)
    x1 = Conv1D(128, 5, activation='relu', padding='same')(x1)
    x1 = BatchNormalization()(x1)
    x1 = MaxPooling1D(2)(x1)
    x1 = Conv1D(256, 3, activation='relu', padding='same')(x1)
    x1 = BatchNormalization()(x1)
    branch1 = GlobalAveragePooling1D()(x1)

    def stft_layer(x):
        x = tf.squeeze(x, axis=-1) if x.shape[-1] == 1 else x
        stft = tf.signal.stft(x, frame_length=128, frame_step=64, fft_length=128)
        mag = tf.abs(stft)
        return mag
    x2 = Lambda(stft_layer)(inputs)
    x2 = Reshape((x2.shape[1], x2.shape[2], 1))(x2)
    x2 = Conv2D(64, (3, 3), activation='relu', padding='same')(x2)
    x2 = BatchNormalization()(x2)
    x2 = MaxPooling2D((2, 2))(x2)
    x2 = Conv2D(128, (3, 3), activation='relu', padding='same')(x2)
    x2 = BatchNormalization()(x2)
    x2 = MaxPooling2D((2, 2))(x2)
    branch2 = GlobalAveragePooling2D()(x2)

    def stats_layer(x):
        mean = tf.reduce_mean(x, axis=1, keepdims=True)
        std = tf.math.reduce_std(x, axis=1, keepdims=True)
        var = tf.math.reduce_variance(x, axis=1, keepdims=True)
        max_val = tf.reduce_max(x, axis=1, keepdims=True)
        min_val = tf.reduce_min(x, axis=1, keepdims=True)
        return tf.concat([mean, std, var, max_val, min_val], -1)
    x3 = Lambda(stats_layer)(inputs)
    x3 = Dense(64, activation='relu')(x3)
    x3 = BatchNormalization()(x3)
    branch3 = Dense(128, activation='relu')(x3)

    merged = Concatenate()([branch1, branch2, branch3])
    shared = Dense(256, activation='relu')(merged)
    shared = Dropout(0.5)(shared)
    shared = Dense(128, activation='relu')(shared)
    shared = Dropout(0.3)(shared)

    det_feat = Dense(64, activation='relu')(shared)
    cls_feat = Dense(64, activation='relu')(shared)
    reg_feat = Dense(64, activation='relu')(shared)

    det_out = Dense(1, activation='sigmoid', name='detection_output')(det_feat)
    cls_out = Dense(num_classes, activation='softmax', name='classification_output')(cls_feat)
    
    # 回归分支：输出归一化参数 ∈ [0,1]
    reg_out = Dense(3, activation='sigmoid', name='regression_output')(reg_feat)  # 使用sigmoid确保输出在[0,1]
    
    model = Model(inputs, [det_out, cls_out, reg_out])
    
    # 编译配置：调整损失权重适应归一化参数
    model.compile(
        optimizer=Adam(1e-3),
        loss={
            'detection_output': BinaryCrossentropy(),
            'classification_output': SparseCategoricalCrossentropy(),
            'regression_output': MeanSquaredError()
        },
        loss_weights={
            'detection_output': 0.8,
            'classification_output': 2.0,
            'regression_output': 0.5  # 提高回归权重，因参数范围已归一化
        },
        metrics={
            'detection_output': BinaryAccuracy(),
            'classification_output': SparseCategoricalAccuracy(),
            'regression_output': 'mae'
        }
    )
    return model

# ---------- 训练（适配v3归一化） ----------
def train_single_model(data, model_idx, epochs=120, batch=128):
    input_shape, num_classes = (data["L"],), len(data["type2label"])
    model = build_model(input_shape, num_classes)

    unique_labels = np.unique(data['y_type_train'])
    cls_weights = compute_class_weight('balanced', classes=unique_labels, y=data['y_type_train'])
    cls_weight_dict = {lab: float(w) for lab, w in zip(unique_labels, cls_weights)}
    sample_weights = np.array([cls_weight_dict[lab] for lab in data['y_type_train']])

    def filter_nan(*args):
        x, y = args[0], args[1]
        ok = (tf.reduce_all(tf.math.is_finite(x)) &
              tf.math.is_finite(y['detection_output']) &
              tf.reduce_all(tf.math.is_finite(y['regression_output'])))
        if len(args) == 3:
            ok &= tf.math.is_finite(args[2])
        return ok

    ds_train = tf.data.Dataset.from_tensor_slices((
        data['X_train'],
        {
            'detection_output': data['y_det_train'],
            'classification_output': data['y_type_train'],
            'regression_output': data['y_param_train']  # 已经是归一化值
        },
        sample_weights
    ))
    ds_train = ds_train.filter(filter_nan)
    ds_train = ds_train.map(lambda x, y, w: (aug_fn_with_mask(x), y, w), num_parallel_calls=tf.data.AUTOTUNE)
    ds_train = ds_train.shuffle(10000).batch(batch).prefetch(tf.data.AUTOTUNE)

    ds_val = tf.data.Dataset.from_tensor_slices((
        data['X_val'],
        {
            'detection_output': data['y_det_val'],
            'classification_output': data['y_type_val'],
            'regression_output': data['y_param_val']  # 已经是归一化值
        }
    ))
    ds_val = ds_val.filter(filter_nan)
    ds_val = ds_val.batch(batch).prefetch(tf.data.AUTOTUNE)

    os.makedirs("models", exist_ok=True)
    ckpt = f"models/combo_aug_model_v3_{model_idx}.keras"
    callbacks = [
        EarlyStopping(monitor='val_classification_output_sparse_categorical_accuracy', patience=20,
                      restore_best_weights=True, mode='max', verbose=1),
        ReduceLROnPlateau(monitor='val_classification_output_sparse_categorical_accuracy', factor=0.5,
                          patience=10, min_lr=1e-5, mode='max', verbose=1),
        ModelCheckpoint(ckpt, save_best_only=True, save_weights_only=False,
                        monitor='val_classification_output_sparse_categorical_accuracy', mode='max', verbose=1)
    ]

    print(f"\n🔥 训练第 {model_idx + 1} 个模型（适配v3归一化数据）...")
    start_time = time.time()
    history = model.fit(ds_train, validation_data=ds_val, epochs=epochs, callbacks=callbacks, verbose=1)
    end_time = time.time()
    training_time = end_time - start_time
    print(f"✅ 模型 {model_idx + 1} 训练完成，耗时 {training_time:.2f} 秒")
    return model, training_time

def train_ensemble(data, n_models=3, epochs=120, batch=128):
    models, training_times = [], []
    for i in range(n_models):
        model, t = train_single_model(data, i, epochs=epochs, batch=batch)
        models.append(model)
        training_times.append(t)
    return models, training_times

# ---------- 评估 + 打印（适配v3归一化） ----------
def evaluate_and_print(models, data, batch=128):
    os.makedirs("reports", exist_ok=True)
    ds_test = tf.data.Dataset.from_tensor_slices((
        data['X_test'],
        {
            'detection_output': data['y_det_test'],
            'classification_output': data['y_type_test'],
            'regression_output': data['y_param_test']  # 归一化值
        }
    )).batch(batch).prefetch(tf.data.AUTOTUNE)

    y_det_true, y_type_true, y_param_true = [], [], []
    y_det_pred, y_type_pred, y_param_pred = [], [], []
    x_input = []

    for x, y in ds_test:
        x_input.append(x.numpy())
        y_det_true.append(y['detection_output'].numpy())
        y_type_true.append(y['classification_output'].numpy())
        y_param_true.append(y['regression_output'].numpy())

        preds = [m(x, training=False) for m in models]
        det_avg = np.mean([p[0].numpy() for p in preds], axis=0)
        cls_avg = np.mean([p[1].numpy() for p in preds], axis=0)
        reg_avg = np.mean([p[2].numpy() for p in preds], axis=0)

        y_det_pred.append(det_avg)
        y_type_pred.append(cls_avg)
        y_param_pred.append(reg_avg)

    x_input = np.concatenate(x_input, axis=0)
    y_det_true = np.concatenate(y_det_true, axis=0)
    y_type_true = np.concatenate(y_type_true, axis=0)
    y_param_true = np.concatenate(y_param_true, axis=0)
    y_det_pred = np.concatenate(y_det_pred, axis=0)
    y_type_pred = np.concatenate(y_type_pred, axis=0)
    y_param_pred = np.concatenate(y_param_pred, axis=0)

    # 反归一化参数用于显示和评估
    y_param_true_denorm = denormalize_params(y_param_true, data['param_stats'], data['jnr_range'])
    y_param_pred_denorm = denormalize_params(y_param_pred, data['param_stats'], data['jnr_range'])

    # 打印前 5 条（显示反归一化后的参数）
    print("\n" + "="*80)
    print("📥 输入信号（前 5 条）")
    print("="*80)
    for i in range(5):
        print(f"样本 {i}: {x_input[i][:10]} ... (len={len(x_input[i])})")

    print("\n" + "="*80)
    print("📤 真实 vs 预测（前 5 条）- 反归一化显示")
    print("="*80)
    for i in range(5):
        print(f"样本 {i}:")
        print(f"  det_true: {y_det_true[i]:.4f}  |  det_pred: {y_det_pred[i,0]:.4f}")
        print(f"  cls_true: {y_type_true[i]}     |  cls_pred: {np.argmax(y_type_pred[i])}")
        print(f"  reg_true: {y_param_true_denorm[i]}  |  reg_pred: {y_param_pred_denorm[i]}")

    # 使用反归一化后的参数计算NRMSE
    def nrmse(y_true, y_pred):
        rmse = np.sqrt(np.mean((y_true - y_pred) ** 2, axis=0))
        y_range = np.max(y_true, axis=0) - np.min(y_true, axis=0)
        return rmse / (y_range + 1e-8)

    reg_nrmse = nrmse(y_param_true_denorm, y_param_pred_denorm)
    print("\n" + "="*80)
    print("📈 NRMSE 报告（基于反归一化参数）")
    print("="*80)
    print("参数维度: [start_time(ms), end_time(ms), jnr_db]")
    for i, name in enumerate(["start_time", "end_time", "jnr_db"]):
        print(f"{name}: {reg_nrmse[i]:.4f}")
    print(f"平均 NRMSE: {np.mean(reg_nrmse):.4f}")

    # 保存评估报告
    with open("reports/nrmse_report_v3.txt", "w", encoding="utf-8") as f:
        f.write("参数维度: [start_time(ms), end_time(ms), jnr_db]\n")
        for i, name in enumerate(["start_time", "end_time", "jnr_db"]):
            f.write(f"{name}: {reg_nrmse[i]:.4f}\n")
        f.write(f"平均 NRMSE: {np.mean(reg_nrmse):.4f}\n")
    print("✅ NRMSE 报告已写入 reports/nrmse_report_v3.txt")

# ---------- 主函数 ----------
def main():
    for d in ["models", "visualizations", "reports"]:
        os.makedirs(d, exist_ok=True)

    print("=" * 80)
    print("🚀 开始训练组合干扰检测与分类模型（适配v3归一化数据）")
    print("=" * 80)

    dataset = load_dataset()
    data = preprocess_data(dataset, aug_ratio=0.1)

    n_models = 3
    epochs = 120
    batch_size = 128

    models, training_times = train_ensemble(data, n_models=n_models, epochs=epochs, batch=batch_size)

    print("\n" + "="*50)
    print("📈 训练时间统计")
    print("="*50)
    for i, t in enumerate(training_times):
        print(f"模型 {i + 1}: {t:.2f} 秒 ({t / 60:.2f} 分钟)")
    print(f"🔥 总训练时间: {sum(training_times):.2f} 秒 ({sum(training_times) / 60:.2f} 分钟)")
    print("="*50)

    print("\n" + "="*80)
    print("🔍 开始评估并打印输入/输出（前 5 条）与 NRMSE")
    print("="*80)
    evaluate_and_print(models, data, batch=batch_size)

    print("\n✅ 所有流程完成！模型已保存至 models/combo_aug_model_v3_{i}.keras")
    print("📌 适配v3归一化数据的训练完成")

if __name__ == "__main__":
    main()

✅ 成功设置中文字体: ['WenQuanYi Micro Hei']
🚀 开始训练组合干扰检测与分类模型（适配v3归一化数据）


/root/miniconda3/lib/python3.8/site-packages/numpy/core/_methods.py:233: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


⚠️ 未找到 metadata，使用默认参数标签
🧹 丢弃 0 / 81000 条含 NaN/Inf 的样本

🔥 训练第 1 个模型（适配v3归一化数据）...
Epoch 1/120


2025-10-19 23:03:02.264567: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype float and shape [56700,3]
	 [[{{node Placeholder/_3}}]]
2025-10-19 23:03:02.264822: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype float and shape [56700,3]
	 [[{{node Placeholder/_3}}]]


    443/Unknown - 13s 17ms/step - loss: 2.3510 - detection_output_loss: 0.1879 - classification_output_loss: 1.0991 - regression_output_loss: 0.0049 - detection_output_binary_accuracy: 0.9014 - classification_output_sparse_categorical_accuracy: 0.5956 - regression_output_mae: 0.0470

...

    443/Unknown - 13s 17ms/step - loss: 2.3536 - detection_output_loss: 0.1836 - classification_output_loss: 1.1021 - regression_output_loss: 0.0052 - detection_output_binary_accuracy: 0.9049 - classification_output_sparse_categorical_accuracy: 0.5966 - regression_output_mae: 0.0469

2025-10-19 23:34:11.992622: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [8100,1024]
	 [[{{node Placeholder/_0}}]]



Epoch 1: val_classification_output_sparse_categorical_accuracy improved from -inf to 0.53198, saving model to models/combo_aug_model_v3_2.keras
443/443 [==============================] - 14s 19ms/step - loss: 2.3536 - detection_output_loss: 0.1836 - classification_output_loss: 1.1021 - regression_output_loss: 0.0052 - detection_output_binary_accuracy: 0.9049 - classification_output_sparse_categorical_accuracy: 0.5966 - regression_output_mae: 0.0469 - val_loss: 3.8285 - val_detection_output_loss: 0.8020 - val_classification_output_loss: 1.5931 - val_regression_output_loss: 0.0014 - val_detection_output_binary_accuracy: 0.8889 - val_classification_output_sparse_categorical_accuracy: 0.5320 - val_regression_output_mae: 0.0229 - lr: 0.0010
Epoch 2/120
443/443 [==============================] - ETA: 0s - loss: 1.6500 - detection_output_loss: 0.1391 - classification_output_loss: 0.7692 - regression_output_loss: 7.3998e-04 - detection_output_binary_accuracy: 0.9267 - classification_output_sp

2025-10-19 23:51:04.654065: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype float and shape [16200,3]
	 [[{{node Placeholder/_3}}]]



📥 输入信号（前 5 条）
样本 0: [-1.05697181  0.00378731  0.42925926  1.55656698  1.4207378   0.19368606
 -0.78710616 -0.63801489  0.52385147  0.51182554] ... (len=1024)
样本 1: [-0.62605659 -0.29576411 -0.14456531  0.57068346  0.50487261 -0.75098009
  0.19402654  0.6368223   0.15793518 -0.01252269] ... (len=1024)
样本 2: [ 1.16228029 -0.28318971  0.57227453 -0.92191901  0.79554124 -0.62132521
  0.8306696  -0.64934691  0.70676081 -0.51393337] ... (len=1024)
样本 3: [-0.42974032  1.24314353 -1.7718107   0.62841242  0.2246237   0.59423203
 -0.6861552  -0.37785159 -1.3150014   0.85562484] ... (len=1024)
样本 4: [-0.28479713 -0.15391934  0.26693762 -0.05293526 -0.55105313 -1.39802339
  0.43218271 -1.03412694  0.05577236 -0.39820111] ... (len=1024)

📤 真实 vs 预测（前 5 条）- 反归一化显示
样本 0:
  det_true: 1.0000  |  det_pred: 1.0000
  cls_true: 2     |  cls_pred: 2
  reg_true: [0.0512 0.0512 0.    ]  |  reg_pred: [ 0.05119759  0.05119657 -0.00059986]
样本 1:
  det_true: 1.0000  |  det_pred: 1.0000
  cls_true: 7     |  cls_p

In [ ]:
# evaluate_models_nrmse_columnwise.py
import os
import json
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (confusion_matrix, accuracy_score, precision_score,
                             recall_score, f1_score, mean_absolute_error)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from matplotlib import font_manager as fm
from tensorflow.keras.layers import Dense
import time

# ---------------------- 中文字体 ----------------------
def set_chinese_font():
    try:
        font_paths = ['/usr/share/fonts/truetype/wqy/wqy-microhei.ttc',
                      'C:/Windows/Fonts/simhei.ttf',
                      '/System/Library/Fonts/PingFang.ttc']
        for font_path in font_paths:
            if os.path.exists(font_path):
                fm.fontManager.addfont(font_path)
                plt.rcParams['font.family'] = fm.FontProperties(fname=font_path).get_name()
                plt.rcParams['axes.unicode_minus'] = False
                print(f"✅ 成功设置中文字体: {plt.rcParams['font.family']}")
                return True
        plt.rcParams['font.family'] = ['SimHei', 'Arial Unicode MS']
        plt.rcParams['axes.unicode_minus'] = False
        print("⚠️ 未找到指定字体，使用默认兼容字体")
        return True
    except Exception as e:
        print(f"❌ 字体设置失败: {e}")
        return False
set_chinese_font()

# ---------------------- 自定义层 ----------------------
class AdaptiveCrossStitch(tf.keras.layers.Layer):
    def __init__(self, num_tasks=3, **kwargs):
        super().__init__(**kwargs)
        self.num_tasks = num_tasks

    def build(self, input_shape):
        feature_dims = [shape[-1] for shape in input_shape]
        if len(set(feature_dims)) != 1:
            raise ValueError("特征维度必须一致")
        self.alpha = self.add_weight(
            name='cross_stitch_alpha',
            shape=(self.num_tasks, self.num_tasks),
            initializer='identity',
            trainable=True
        )
        self.gate = Dense(self.num_tasks, activation='sigmoid')
        super().build(input_shape)

    def call(self, inputs):
        concat = tf.concat(inputs, axis=-1)
        gate = self.gate(concat)
        outputs = []
        for i in range(self.num_tasks):
            weighted = tf.zeros_like(inputs[0])
            for j in range(self.num_tasks):
                weighted += tf.expand_dims(gate[:, i], -1) * self.alpha[i, j] * inputs[j]
            outputs.append(weighted)
        return outputs

# ---------------------- 加载数据集 ----------------------
def load_dataset(npz_path="/root/yxun/20250826/dataset/interference_signals_natural_same_freq_1019.npz"):
    data = np.load(npz_path, allow_pickle=True)
    signals = data["signals"]
    labels = data["labels"].astype(np.int32)
    jnr_vals = data["jnr_values"].astype(np.float32)
    fs = float(data["fs"])
    L = int(data["L"])
    metadata = data["metadata"]
    type2label = data["type_to_label"].item()
    label2type = {v: k for k, v in type2label.items()}
    type2name = data["interference_type_names"].item()
    label2name = {i: type2name[k] for k, i in type2label.items()}
    return {
        "signals": signals,
        "labels": labels,
        "jnr_values": jnr_vals,
        "fs": fs,
        "L": L,
        "metadata": metadata,
        "type2label": type2label,
        "label2name": label2name,
        "type2name": type2name
    }

# ---------------------- 数据预处理 ----------------------
# ---------------------- 数据预处理 ----------------------
def preprocess_data(dataset):
    signals = dataset["signals"]
    labels = dataset["labels"]
    jnr_values = dataset["jnr_values"]
    L = dataset["L"]

    signals = np.array([StandardScaler().fit_transform(s.reshape(-1, 1)).ravel() for s in signals])

    no_key = "satellite_signal"
    det_labels = (labels != dataset["type2label"][no_key]).astype(np.float32)

    # 使用生成时记录的参数作为标签 - 修复None值处理
    param_labels = []
    for m in dataset["metadata"]:
        p = m.get("params", {})
        # 安全地处理可能为None的参数值
        start_time = p.get("start_time", 0)
        end_time = p.get("end_time", 0)
        jnr_db = p.get("jnr_db", 0)
        
        # 处理None值，将其转换为0或其他默认值
        start_time = float(start_time) if start_time is not None else 0.0
        end_time = float(end_time) if end_time is not None else 0.0
        jnr_db = float(jnr_db) if jnr_db is not None else 0.0
        
        param_labels.append([start_time, end_time, jnr_db])
    param_labels = np.array(param_labels, dtype=np.float32)

    X_test = signals
    y_det_test = det_labels
    y_type_test = labels
    y_param_test = param_labels
    jnr_test = jnr_values
    label2name = dataset["label2name"]

    return {
        "X_test": X_test,
        "y_det_test": y_det_test,
        "y_type_test": y_type_test,
        "y_param_test": y_param_test,
        "jnr_values_test": jnr_test,
        "label2name": label2name,
        "L": L
    }

# ---------------------- 加载模型并预测 ----------------------
def load_and_predict(model_paths, X_test):
    all_det, all_cls, all_reg = [], [], []
    for path in model_paths:
        print(f"🔁 加载模型: {path}")
        model = tf.keras.models.load_model(
            path,
            custom_objects={'AdaptiveCrossStitch': AdaptiveCrossStitch}
        )
        start_time = time.time()
        det, cls, reg = model.predict(X_test, verbose=0)
        end_time = time.time()
        prediction_time = end_time - start_time
        print(f"预测时间: {prediction_time:.2f} 秒")
        all_det.append(det)
        all_cls.append(cls)
        all_reg.append(reg)
    avg_det = np.mean(all_det, axis=0) > 0.5
    avg_cls = np.argmax(np.mean(all_cls, axis=0), axis=1)
    avg_reg = np.mean(all_reg, axis=0)
    return avg_det, avg_cls, avg_reg

# ---------------------- 绘制混淆矩阵 ----------------------
def plot_confusion_matrix(cm, labels, title, xlabel, ylabel, filename, dpi=150, rotate_x=False):
    cm_normalized = cm.astype('float') / cm.sum(axis=1, keepdims=True)
    cm_normalized = np.nan_to_num(cm_normalized)

    plt.figure(figsize=(12, 10))
    ax = sns.heatmap(cm_normalized,
                     annot=True,
                     fmt='.2f',
                     cmap='Blues',
                     xticklabels=labels,
                     yticklabels=labels,
                     square=True,
                     annot_kws={"size": 14})
    cbar = ax.collections[0].colorbar
    cbar.ax.tick_params(labelsize=14)

    plt.title(title, pad=20, fontsize=18)
    plt.xlabel(xlabel, fontsize=16)
    plt.ylabel(ylabel, fontsize=16)
    plt.xticks(rotation=45 if rotate_x else 0, ha='right' if rotate_x else 'center', fontsize=14)
    plt.yticks(rotation=0, fontsize=14)
    plt.tight_layout()
    plt.savefig(filename, dpi=dpi)
    plt.close()

# ... existing code ...
# ---------------------- 按列分别归一化 NRMSE ----------------------
def nrmse_columnwise(y_true, y_pred):
    rmse = np.sqrt(np.mean((y_true - y_pred) ** 2, axis=0))  # shape (3,)
    y_range = np.max(y_true, axis=0) - np.min(y_true, axis=0)
    return rmse / (y_range + 1e-8)

# ---------------------- 优化后的评估函数 ----------------------
def evaluate(models_dir="models",
             npz_path="/root/yxun/20250826/dataset/interference_signals_natural_same_freq_1019.npz",
             wanted_jnr=np.arange(-10, 31, 5),
             dpi=150):
    """
    评估模型性能，包括检测、分类和参数估计
    
    Args:
        models_dir: 模型文件夹路径
        npz_path: 数据集路径
        wanted_jnr: 要评估的JNR值列表
        dpi: 图像分辨率
    """
    
    # 创建输出目录
    os.makedirs("visualizations", exist_ok=True)
    os.makedirs("reports", exist_ok=True)
    
    print("⏳ 加载数据集...")
    dataset = load_dataset(npz_path)
    data = preprocess_data(dataset)
    
    # 提取测试数据
    X_test = data["X_test"]
    y_det_test = data["y_det_test"]
    y_type_test = data["y_type_test"]
    y_param_test = data["y_param_test"]
    jnr_test = data["jnr_values_test"]
    label2name = data["label2name"]
    
    # 加载并预测
    model_paths = [os.path.join(models_dir, f"combo_aug_model_v3_{i}.keras") for i in range(3)]
    avg_det, avg_cls, avg_reg = load_and_predict(model_paths, X_test)
    
    # 数据清理：移除NaN和无穷大值
    invalid_mask = (
        np.any(np.isnan(y_param_test), axis=1) | 
        np.any(np.isinf(y_param_test), axis=1) |
        np.any(np.isnan(avg_reg), axis=1) | 
        np.any(np.isinf(avg_reg), axis=1)
    )
    
    valid_y_param_test = y_param_test[~invalid_mask]
    valid_avg_reg = avg_reg[~invalid_mask]
    
    # 1. 检测任务评估
    cm_det = confusion_matrix(y_det_test, avg_det)
    plot_confusion_matrix(
        cm=cm_det,
        labels=['No Interference', 'Interference'],
        title='Detection Confusion Matrix',
        xlabel='Predicted',
        ylabel='True',
        filename='visualizations/detection_confusion_matrix.png',
        dpi=dpi,
        rotate_x=False
    )
    
    det_acc = accuracy_score(y_det_test, avg_det)
    
    # 2. 分类任务评估（仅对干扰信号）
    # 获取干扰信号的索引
    interference_mask = y_det_test == 1
    if np.sum(interference_mask) > 0:
        # 只评估干扰信号的分类
        y_type_interf = y_type_test[interference_mask]
        avg_cls_interf = avg_cls[interference_mask]
        
        cm_type = confusion_matrix(y_type_interf, avg_cls_interf)
        plot_confusion_matrix(
            cm=cm_type,
            labels=[label2name[i] for i in sorted(label2name.keys())],
            title='Classification Confusion Matrix (Interference Only)',
            xlabel='Predicted',
            ylabel='True',
            filename='visualizations/classification_confusion_matrix.png',
            dpi=dpi,
            rotate_x=True
        )
        
        cls_acc = accuracy_score(y_type_interf, avg_cls_interf)
        cls_precision = precision_score(y_type_interf, avg_cls_interf, average='weighted', zero_division=0)
        cls_recall = recall_score(y_type_interf, avg_cls_interf, average='weighted', zero_division=0)
        cls_f1 = f1_score(y_type_interf, avg_cls_interf, average='weighted', zero_division=0)
    else:
        cls_acc = cls_precision = cls_recall = cls_f1 = 0.0
        print("⚠️  测试集中没有干扰信号，无法进行分类评估")
    
    # 3. JNR vs 准确率
    jnr_acc = []
    for jnr in wanted_jnr:
        mask = jnr_test == jnr
        mask = mask & ~invalid_mask
        if np.sum(mask) == 0:
            acc = np.nan
        else:
            acc = accuracy_score(y_type_test[mask], avg_cls[mask])
        jnr_acc.append(acc)
    
    plt.figure(figsize=(8, 5))
    valid_mask = ~np.isnan(jnr_acc)
    plt.plot(wanted_jnr[valid_mask], np.array(jnr_acc)[valid_mask], marker='o', linewidth=2)
    if np.any(~valid_mask):
        plt.scatter(wanted_jnr[~valid_mask], [1.0] * np.sum(~valid_mask),
                    facecolors='none', edgecolors='r', s=60)
    plt.xlabel('JNR (dB)', fontsize=14)
    plt.ylabel('Accuracy', fontsize=14)
    plt.title('Classification Accuracy vs JNR', fontsize=16)
    plt.xticks(wanted_jnr)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.ylim(0, 1.05)
    plt.tight_layout()
    plt.savefig('visualizations/jnr_vs_accuracy.png', dpi=dpi)
    plt.close()
    
    # 4. 参数估计评估
    param_mae = mean_absolute_error(valid_y_param_test, valid_avg_reg, multioutput='raw_values')
    param_names = ['Start Time (ms)', 'End Time (ms)', 'JNR (dB)']
    
    # 按列分别归一化 NRMSE
    def nrmse_columnwise(y_true, y_pred):
        rmse = np.sqrt(np.mean((y_true - y_pred) ** 2, axis=0))
        y_range = np.max(y_true, axis=0) - np.min(y_true, axis=0)
        return rmse / (y_range + 1e-8)
    
    param_nrmse = nrmse_columnwise(valid_y_param_test, valid_avg_reg)
    
    # 5. 输出评估结果
    print("\n" + "="*50)
    print("📊 评估结果")
    print("="*50)
    print(f"检测准确率: {det_acc:.4f}")
    print(f"分类准确率: {cls_acc:.4f}")
    print(f"分类精确率: {cls_precision:.4f}, 召回率: {cls_recall:.4f}, F1: {cls_f1:.4f}")
    print("\n参数估计误差（MAE & 按列 NRMSE）:")
    for i, (name, mae, nrmse) in enumerate(zip(param_names, param_mae, param_nrmse)):
        print(f"  {name}: MAE = {mae:.4f}, NRMSE = {nrmse:.4f}")
    print(f"  平均 NRMSE（三列分别归一化） = {np.mean(param_nrmse):.4f}")
    
    print("\nJNR 准确率:")
    for j, acc in zip(wanted_jnr, jnr_acc):
        print(f"  {int(j)}dB: {acc if not np.isnan(acc) else 'N/A'}")
    
    # 6. 保存报告
    report = {
        "detection_accuracy": float(det_acc),
        "classification_accuracy": float(cls_acc),
        "classification_precision": float(cls_precision),
        "classification_recall": float(cls_recall),
        "classification_f1": float(cls_f1),
        "parameter_mae": [float(m) for m in param_mae],
        "parameter_nrmse": [float(n) for n in param_nrmse],
        "average_columnwise_nrmse": float(np.mean(param_nrmse)),
        "parameter_details": {
            param_names[i]: {"mae": float(param_mae[i]), "nrmse": float(param_nrmse[i])}
            for i in range(len(param_names))
        },
        "jnr_accuracies": {f"{int(j)}dB": float(acc) if not np.isnan(acc) else None for j, acc in zip(wanted_jnr, jnr_acc)},
        "invalid_samples_count": int(np.sum(invalid_mask)),
        "total_samples": len(X_test),
        "interference_samples": int(np.sum(y_det_test))
    }
    
    with open("reports/evaluation_report.json", "w", encoding='utf-8') as f:
        json.dump(report, f, indent=4, ensure_ascii=False)
    
    print("\n✅ 评估完成！混淆矩阵与报告已保存。")
    
# ---------------------- 主函数 ----------------------
if __name__ == "__main__":
    evaluate(models_dir="models",
             npz_path="/root/yxun/20250826/dataset/interference_signals_natural_same_freq_1019.npz",
             wanted_jnr=np.arange(-10, 31, 5),
             dpi=150)

2025-10-21 16:54:48.541760: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-21 16:54:48.581358: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-21 16:54:49.143543: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


✅ 成功设置中文字体: ['WenQuanYi Micro Hei']
⏳ 加载数据集...
🔁 加载模型: models/combo_aug_model_v3_0.keras


2025-10-21 16:55:09.192052: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 890 MB memory:  -> device: 0, name: NVIDIA vGPU-48GB, pci bus id: 0000:d8:00.0, compute capability: 8.9
2025-10-21 16:55:10.433761: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2025-10-21 16:55:10.442767: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


预测时间: 10.85 秒
🔁 加载模型: models/combo_aug_model_v3_1.keras
预测时间: 10.30 秒
🔁 加载模型: models/combo_aug_model_v3_2.keras


✅ 成功设置中文字体: ['WenQuanYi Micro Hei']
🔁 加载模型: models/combo_aug_model_v3_0.keras
预测时间: 9.99 秒
🔁 加载模型: models/combo_aug_model_v3_1.keras
预测时间: 10.55 秒
🔁 加载模型: models/combo_aug_model_v3_2.keras
预测时间: 10.35 秒

==================================================
📊 评估结果
==================================================
检测准确率: 0.9813
分类准确率: 0.9109
分类精确率: 0.9175, 召回率: 0.9109, F1: 0.9103

参数估计误差（MAE & 按列 NRMSE）:
  Start Time (ms): MAE = 0.3839, NRMSE = 0.4120
  End Time (ms): MAE = 0.3840, NRMSE = 0.4105
  JNR (dB): MAE = 13.2500, NRMSE = 0.4045
  平均 NRMSE（三列分别归一化） = 0.4090

JNR 准确率:
  -10dB: 0.653125
  -5dB: 0.77925
  0dB: 0.8715
  5dB: 0.913375
  10dB: 0.94525
  15dB: 0.976
  20dB: 0.9885
  25dB: 0.99025
  30dB: 0.9895